In [1]:
# !pip install linearmodels # uncomment if linear models is not installed
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from IPython.display import display, Markdown

In [2]:
class random_coeff:
    def __init__(self,
                m=196, # number of markets
                r=4, # radius
                bl=1.25, # beta_0^local= intercept of random coefficient for local brewery
                bc=0.75, # beta_0^corporate= intercept of random coefficient for corporate brewery
                bs=1, # beta_0^subsidiary= intercept of random coefficient for subsidiary brewery
                ba=1, # beta^alcohol = constant coefficient on alcohol
                bb=1, # beta^bitterness = constant coefficient on bitterness
                alpha = 1, # coefficient on price
                b1l = 0.8, # beta_1^local= slope of random coefficient for local brewery
                b1c = 0.8, # beta_1^corporate= slope of random coefficient for corporate brewery
                b1s = 0.8, # beta_1^subsidiary= slope of random coefficient for subsidiary brewery
                n_ind = 200 # number of consumers for heterogeneity
                ):
                      
        # Save parameters
        self.m, self.r,self.bl, self.bc, self.bs, self.ba, self.bb, self.alpha = m,r,bl,bc,bs,ba,bb,alpha
        self.b1l, self.b1c, self.b1s, self.n_ind = b1l, b1c, b1s, n_ind

    
    # main function of class random_coef - using functions defined below, it gives the data according to random coefficient model
    def data_gen(self):
        market = np.arange(1, np.sqrt(self.m)*np.sqrt(self.m)+1).reshape(int(np.sqrt(self.m)), int(np.sqrt(self.m))) # create a 14 * 14 matrix with market number

        # allocate local breweries so that each market has at leat two local beers using r = 4: I choose markets 1, 10, 61, 70, 127, 136, 187, 196
        local_brew = [1,10,61,70,127,136, 187, 196]

        # Create a dataset with firm and market served
        # Initiate a placeholder
        data = []

        # For each local breweries, identify the markets served and Assign into data set
        for brew in local_brew:
            rows,cols = np.where(np.isin(market, brew)) # locate rows and columns where the brew is located
            served_by = [] # place holder to store what are the markets served by it

            for i in range(market.shape[0]):  # Row indices
                for j in range(market.shape[1]):  # Column indices
                    if max(abs(rows - i), abs(cols - j)) <= self.r:
                        served_by.append(market[i, j])
            #globals()[f"served_by_{brew}"] = served_by

            #create dummy variables
            isLocal = 1
            isSubsidiary = 0
            isCorporate = 0

            # generate alcohol and bitterness percentage
            alc = np.random.uniform(0, 1)
            bitt = np.random.uniform(0, 1)

            # unobservables in utility for each firm
            xi_j = np.random.randn()

            # Assign market for local breweries and their alcohol, bitterness, marginal costs
            for market_id in served_by:
                # unobservables in utility for each firm
                #xi_j = np.random.randn()
                #calculate marginal costs
                w_jt = np.random.randn()
                omega_jt = np.random.randn()
                mc_jt = np.exp(w_jt/8)*np.exp(omega_jt/10)
                data.append([f"local_{brew}", market_id, alc, bitt, w_jt, omega_jt, mc_jt, xi_j, isLocal, isSubsidiary, isCorporate])

        # For Corporate Breweries, identify the markets served (all market served by these) and Assign into data set
        for cbrew in ['AB', 'AB2', 'MC', 'MC2', 'BB']:
            #create dummy variables
            isLocal = 0
            isSubsidiary = 1 if cbrew in ['AB2', 'MC2'] else 0
            isCorporate = 1 if cbrew in ['AB', 'MC', 'BB'] else 0

            # alcohol and bitterness percentage
            alc = np.random.uniform(0, 1)
            bitt = np.random.uniform(0, 1)

            # unobservables in utility for each firm
            xi_j = np.random.randn()
            
            for market_id in range(1, self.m + 1):
                # unobservables in utility for each firm
                #xi_j = np.random.randn()
                #calculate marginal cost
                w_jt = np.random.randn()
                omega_jt = np.random.randn()
                mc_jt = 0.8*np.exp(w_jt/8)*np.exp(omega_jt/10)
                data.append([cbrew, market_id, alc, bitt, w_jt, omega_jt, mc_jt, xi_j , isLocal, isSubsidiary, isCorporate])

        # Create the dataframe
        df = pd.DataFrame(data, columns=['firm', 'market', 'alcohol', 'bitterness', 'w_jt', 'omega_jt', 'mc_jt', 'xi_j', 'isLocal', 'isSubsidiary', 'isCorporate'])
        
        # Iteration for finding equilibrium prices
        # Initial guess of price as marginal cost and a placeholder for updated price
        df['guess_price'] = df['mc_jt'].copy()
        df['updated_price'] = df['mc_jt'].copy()

        # Tolerance for convergence
        tolerance = 1e-6
        max_iterations = 1000

        # Iterative convergence process
        for iteration in range(max_iterations):
            # calculate shares for each updates
            df = share(df, self.bl, self.bc, self.bs, self.ba, self.bb, self.alpha, self.b1l, self.b1c, self.b1s, self.n_ind)

            #calculate updated price
            df['updated_price'] = df.apply(lambda row: calculate_new_price(row, df, self.alpha), axis = 1)

            # Check for convergence
            price_difference = np.abs(df['updated_price'] - df['guess_price'])
            # update price for next iteration
            df['guess_price'] = df['updated_price']
            
            if price_difference.max() < tolerance:
                print(f"Converged: {iteration + 1} iterations")
                break
            else:
                print(f"{iteration+1} iterations")

        # Final dataframe and market shares after price convergence
        self.data = share(df, self.bl, self.bc, self.bs, self.ba, self.bb, self.alpha, self.b1l, self.b1c, self.b1s, self.n_ind)
        self.data = alcohol_other_wg(self.data)

        #################### Estimation ##########################
        # Dependent variable (log(s_j) - log(s_o))
        self.data['demand'] = np.log(self.data['s_jt']) - np.log(self.data['s_outside'])
        # log of within group market share
        self.data['log_sjt_wg'] = np.log(self.data['s_jt_wg'])

    
        

##############################################################################################################################
######################################## Define required Functions ###########################################################
##############################################################################################################################
# function to iterate price
def calculate_new_price(row, df, alpha):
    if row['firm'] in ['BB'] or row['isLocal'] == 1:
        return (1/(alpha * (1 - row['s_jt']))) + row['mc_jt']
    elif row['firm'] in ['AB', 'AB2']:
        # Filter the DataFrame for the same market as the current row
        same_market = df[df['market'] == row['market']]

        # Identify the "other" firm(s) -- subsidiary firm in the same market
        other_firms = same_market[(same_market['firm'] != row['firm']) & (same_market['firm'].isin(['AB', 'AB2']))]

        # obtain the "price of other" and "share of other" subsidiary firm
        other_price = other_firms['guess_price'].values[0]
        other_share = other_firms['s_jt'].values[0]
        other_mc = other_firms['mc_jt'].values[0]

        # Return the updated price
        return row['mc_jt'] + ((1 + alpha * (other_price - other_mc)*other_share)/(alpha * (1 - row['s_jt'])))
    elif row['firm'] in ['MC', 'MC2']:
        # Filter the DataFrame for the same market as the current row
        same_market = df[df['market'] == row['market']]
        # Identify the "other" firm(s) -- subsidiary firm in the same market
        other_firms = same_market[(same_market['firm'] != row['firm']) & (same_market['firm'].isin(['MC', 'MC2']))]

        # obtain the "price of other" and "share of other" subsidiary firm
        other_price = other_firms['guess_price'].values[0]
        other_share = other_firms['s_jt'].values[0]
        other_mc = other_firms['mc_jt'].values[0]

        # Return the updated price
        return row['mc_jt'] + ((1 + alpha * (other_price - other_mc)*other_share)/(alpha * (1 - row['s_jt'])))
    else:
        raise ValueError("Unexpected firm value")
        

# function to calculate share
def share(df, bl, bc, bs, ba, bb, alpha, b1l, b1c, b1s, n_ind):
    delta_jt = bl * df['isLocal'] + bc * df['isCorporate'] + bs * df['isSubsidiary'] + ba * df['alcohol'] + bb * df['bitterness'] - alpha * df['updated_price'] + df['xi_j']
    df1 = df.copy()
    s_jt = np.zeros(len(df))
    np.random.seed(78114)
    unique_markets = df['market'].unique()
    for indiv in range(n_ind):
        # Draw nu_it for each group (local, subsidiary, corporate) and market
        nu_it_local = {market: np.random.randn() for market in unique_markets}
        nu_it_corporate = {market: np.random.randn() for market in unique_markets}
        nu_it_subsidiary = {market: np.random.randn() for market in unique_markets}

        # Assign nu_it based on group (local, subsidiary, corporate)
        nu_it = (
            df['market'].map(nu_it_local) * df['isLocal'] +
            df['market'].map(nu_it_corporate) * df['isCorporate'] +
            df['market'].map(nu_it_subsidiary) * df['isSubsidiary']
        )
        eta_ijt = b1l * df['isLocal'] * nu_it + b1c * df['isCorporate'] * nu_it + b1s * df['isSubsidiary'] * nu_it
        u_ijt = delta_jt + eta_ijt
        num = np.exp(u_ijt)
        df1['num'] = num # assign numerator to dataframe
        s_ijt = num/(1 + df1.groupby('market')['num'].transform('sum'))
        s_jt += s_ijt #average
    df['s_jt'] = s_jt/n_ind # market share for each firm

    # Calculate the market share for outside good
    df['s_outside'] = 1 - df.groupby('market')['s_jt'].transform('sum')

    # sum of share for firms in each nest and market
    df['sum_s_jt'] = df.groupby(['market', 'isLocal'])['s_jt'].transform('sum')

    # Calculate the within group market share
    df['s_jt_wg'] = df['s_jt'] / df['sum_s_jt']
    return df

# assign alcohol percentage of other firm within same group (local or not local) to be used as IV
def alcohol_other_wg(df):
    # Initialize a new column for reassigned alcohol percentages
    df['alcohol_other'] = np.nan

    # Assign alcohol percentage of other local firms for each local firms in a market
    local_groups = df[df['isLocal'] == 1].groupby('market')
    for market, group in local_groups:
        values = group['alcohol'].values
        # Shift values deterministically within the group
        reassigned_values = np.roll(values, shift=1)  # Circular shift
        df.loc[group.index, 'alcohol_other'] = reassigned_values

    # Assign alcohol percentage of other non-local firms for each non-local firms in a market
    non_local_groups = df[df['isLocal'] == 0].groupby('market')
    for market, group in non_local_groups:
        values = group['alcohol'].values
        # Shift values deterministically within the group
        reassigned_values = np.roll(values, shift=1)  # Circular shift
        df.loc[group.index, 'alcohol_other'] = reassigned_values
    return df

In [3]:
rc = random_coeff(m=196, # number of markets
                r=4, # radius
                bl=1.25, # beta_0^local= intercept of random coefficient for local brewery
                bc=0.75, # beta_0^corporate= intercept of random coefficient for corporate brewery
                bs=1, # beta_0^subsidiary= intercept of random coefficient for subsidiary brewery
                ba=1, # beta^alcohol = constant coefficient on alcohol
                bb=1, # beta^bitterness = constant coefficient on bitterness
                alpha = 1, # coefficient on price
                b1l = 0.8, # beta_1^local= slope of random coefficient for local brewery
                b1c = 0.8, # beta_1^corporate= slope of random coefficient for corporate brewery
                b1s = 0.8, # beta_1^subsidiary= slope of random coefficient for subsidiary brewery
                n_ind = 200 # number of consumers for heterogeneity
                )
#set seed for replication
np.random.seed(5811)
rc.data_gen()

1 iterations
2 iterations
3 iterations
4 iterations
5 iterations
6 iterations
7 iterations
8 iterations
9 iterations
10 iterations
11 iterations
12 iterations
13 iterations
14 iterations
15 iterations
16 iterations
17 iterations
18 iterations
Converged: 19 iterations


In [4]:
# Question 4: Own and Cross Price derivatives of market shares implied by true model
# Our data from above
data = rc.data
#data.to_excel('data.xlsx')
# Filter for market 1
market_df = data[data['market'] == 1].reset_index(drop=True)
# Extract firm names and shares
firms = market_df['firm'].values
shares = market_df['s_jt'].values
# Initialize the matrix
share_matrix = np.zeros((len(firms), len(firms)))
# Populate the matrix
for i in range(len(firms)):
    for j in range(len(firms)):
        if i == j:
            # Diagonal: -share * (1 - share)
            share_matrix[i, j] = -shares[i] * (1 - shares[i])
        else:
            # Off-diagonal: -share_j * share_k
            share_matrix[i, j] = shares[i] * shares[j]
# Convert to DataFrame for better readability
share_matrix_df = pd.DataFrame(share_matrix, index=firms, columns=firms)
# Export to LaTeX
latex_output = share_matrix_df.to_latex(index=True, header=True, float_format="%.4f")

# Save LaTeX output to file
with open('true.tex', 'w') as f:
    f.write(latex_output)
    
# Display the result
print(f"Share Derivative matrix implied by true model: \n \n{share_matrix_df}")

Share Derivative matrix implied by true model: 
 
           local_1  local_61        AB       AB2        MC       MC2        BB
local_1  -0.179957  0.052471  0.007397  0.037231  0.023099  0.008944  0.029223
local_61  0.052471 -0.173247  0.007008  0.035272  0.021883  0.008474  0.027685
AB        0.007397  0.007008 -0.030444  0.004973  0.003085  0.001195  0.003903
AB2       0.037231  0.035272  0.004973 -0.133172  0.015527  0.006012  0.019644
MC        0.023099  0.021883  0.003085  0.015527 -0.088516  0.003730  0.012188
MC2       0.008944  0.008474  0.001195  0.006012  0.003730 -0.036561  0.004719
BB        0.029223  0.027685  0.003903  0.019644  0.012188  0.004719 -0.108754


In [5]:
# Question 5: Estimate the demand model
# Run Regression to estimate nested logit
result_iv = IV2SLS(data['demand'], data[['isLocal', 'isSubsidiary', 'isCorporate', 'alcohol', 'bitterness']], data[['updated_price', 'log_sjt_wg']], data[['w_jt', 'alcohol_other']]).fit()

In [6]:
print(result_iv)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 demand   R-squared:                      0.7711
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7701
No. Observations:                1372   F-statistic:                    9087.4
Date:                Sun, Dec 08 2024   P-value (F-stat)                0.0000
Time:                        14:06:52   Distribution:                  chi2(7)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
isLocal           1.6958     0.1768     9.5890     0.0000      1.3492      2.0424
isSubsidiary      1.0111     0.4785     

In [7]:
# Question 7: Own and Cross Price Derivatives of market shares implied by estimated model
# Estimated parameters:
alpha_hat = np.abs(result_iv.params['updated_price']) # coefficient on price
phi_hat = np.abs(result_iv.params['log_sjt_wg'])# nesting parameter
# Filter for market 1
market_df = data[data['market'] == 1].reset_index(drop=True)
# Extract firm names and shares
firms = market_df['firm'].values
shares = market_df['s_jt'].values
is_local = market_df['isLocal'].values
share_within = market_df['s_jt_wg'].values

# Initialize the matrix
share_matrix = np.zeros((len(firms), len(firms)))

# Populate the matrix
for i in range(len(firms)):
    for j in range(len(firms)):
        if i == j:
            # Diagonal:
            share_matrix[i, j] = -(alpha_hat / (1 - phi_hat)) * shares[i] * (1 - phi_hat * share_within[i] - (1 - phi_hat) * shares[i])
        else:
            if is_local[i] == is_local[j]:  # Same group (both local or both non-local)
                # Off-diagonal (same group): - alpha_hat * share_j * share_k * (1 + (phi_hat/(1 - phi_hat)) * (share_j_within / share_j))
                share_matrix[i, j] = alpha_hat * shares[i] * shares[j] * (1 + (phi_hat / (1 - phi_hat)) * (share_within[i] / shares[i]))
            else:
                # Off-diagonal (different group): -alpha * share_j * share_k
                share_matrix[i, j] = alpha_hat * shares[i] * shares[j]

# Convert to DataFrame for better readability
share_matrix_df = pd.DataFrame(share_matrix, index=firms, columns=firms)
# Export to LaTeX
latex_output = share_matrix_df.to_latex(index=True, header=True, float_format="%.4f")

# Save LaTeX output to file
with open('estimated_matrix.tex', 'w') as f:
    f.write(latex_output)

print(share_matrix_df)

           local_1  local_61        AB       AB2        MC       MC2        BB
local_1  -0.180896  0.073685  0.006221  0.031310  0.019425  0.007522  0.024576
local_61  0.073685 -0.175253  0.005893  0.029662  0.018403  0.007126  0.023282
AB        0.006221  0.005893 -0.033151  0.007035  0.004365  0.001690  0.005522
AB2       0.031310  0.029662  0.007035 -0.138476  0.021967  0.008506  0.027791
MC        0.019425  0.018403  0.004365  0.021967 -0.094252  0.005277  0.017242
MC2       0.007522  0.007126  0.001690  0.008506  0.005277 -0.039730  0.006676
BB        0.024576  0.023282  0.005522  0.027791  0.017242  0.006676 -0.114669


In [8]:
# Question 8: Simulating post-buyout prices treating firms marginal costs as known
# use the product characteristics from above as the product characterisitcs does not change after buyout
# assume local brewery at 1st market was acquired by BB i.e local_1 is BB
data_buyout = data[['firm', 'market', 'alcohol', 'bitterness', 'isLocal', 'isSubsidiary', 'isCorporate', 'xi_j', 'w_jt', 'omega_jt']]

# Modify the DataFrame based on the conditions that local_1 is acquired and becomes BB2
data_buyout.loc[data_buyout['firm'] == 'local_1', 'firm'] = 'BB2' # change name
data_buyout.loc[data_buyout['firm'] == 'BB2', 'isLocal'] = 0 # not local
data_buyout.loc[data_buyout['firm'] == 'BB2', 'isSubsidiary'] = 1 # subsidiary

# Extend the market it serves to all 196
# Get the current markets served by BB2 (before buyout)
current_markets = data_buyout.loc[data_buyout['firm'] == 'BB2', 'market'].unique()

# Find the missing markets (after buyout)
missing_markets = set(range(1, 197)) - set(current_markets)

# Get values for `alcohol`, `bitterness`, and unobserved characters (they do not change)
alc = data_buyout.loc[data_buyout['firm'] == 'BB2', 'alcohol'].iloc[0]
bitter = data_buyout.loc[data_buyout['firm'] == 'BB2', 'bitterness'].iloc[0]
xi = data_buyout.loc[data_buyout['firm'] == 'BB2', 'xi_j'].iloc[0]

np.random.seed(48711)
# Create new rows for the missing markets, fill omega_jt and w_jt from standard normal distribution
new_rows = [{'firm': 'BB2', 'market': m, 'alcohol': alc, 'bitterness': bitter,
             'isLocal': 0, 'isCorporate': 0, 'isSubsidiary': 1, 'xi_j' : xi, 'w_jt' : np.random.randn(), 'omega_jt': np.random.randn()} for m in missing_markets]

# Append new rows to the DataFrame
data_buyout = pd.concat([data_buyout, pd.DataFrame(new_rows)], ignore_index=True)


##############################################################################################################################
################################### Marginal Costs Assuming BB2 did not adopt BB's cost structure ############################
##############################################################################################################################
data_buyout['mc_jt'] = np.where(
    (data_buyout['isLocal'] == 1) | (data_buyout['firm'] == 'BB2'),
    np.exp(data_buyout['w_jt'] / 8) * np.exp(data_buyout['omega_jt'] / 10),
    0.8 * np.exp(data_buyout['w_jt'] / 8) * np.exp(data_buyout['omega_jt'] / 10)
)

##############################################################################################################################
################################### Marginal Costs Assuming BB2 adopt BB's cost structure ####################################
##############################################################################################################################
data_buyout['mc_jt1'] = np.where(
    (data_buyout['isLocal'] == 1),
    np.exp(data_buyout['w_jt'] / 8) * np.exp(data_buyout['omega_jt'] / 10),
    0.8 * np.exp(data_buyout['w_jt'] / 8) * np.exp(data_buyout['omega_jt'] / 10)
)



In [ ]:
##############################################################################################################################
########################################### Post Buyout price iteration using Nested Logit FOCs ##############################
##############################################################################################################################
# function to iterate price
def calculate_new_price(row, df, alpha_hat, phi_hat, mc):
    if row['isLocal'] == 1:
        return row[mc] + (1 - phi_hat)/(alpha_hat * (1 - phi_hat * row['s_jt_wg'] - (1 - phi_hat) * row['s_jt']))
    elif row['firm'] in ['AB', 'AB2']:
        # Filter the DataFrame for the same market as the current row
        same_market = df[df['market'] == row['market']]

        # Identify the "other" firm(s) -- subsidiary firm in the same market
        other_firms = same_market[(same_market['firm'] != row['firm']) & (same_market['firm'].isin(['AB', 'AB2']))]

        # obtain the "price of other" and "share of other" subsidiary firm
        other_price = other_firms['guess_price'].values[0]
        other_share = other_firms['s_jt'].values[0]
        other_mc = other_firms[mc].values[0]

        # Return the updated price
        return row[mc] + ((1 - phi_hat)/(alpha_hat * (1 - phi_hat * row['s_jt_wg'] - (1 - phi_hat) * row['s_jt']))) * (1 + alpha_hat * other_share * (other_price - other_mc) * (1 + (phi_hat * row['s_jt_wg'])/((1 - phi_hat)*row['s_jt'])))
    elif row['firm'] in ['MC', 'MC2']:
        # Filter the DataFrame for the same market as the current row
        same_market = df[df['market'] == row['market']]
        # Identify the "other" firm(s) -- subsidiary firm in the same market
        other_firms = same_market[(same_market['firm'] != row['firm']) & (same_market['firm'].isin(['MC', 'MC2']))]

        # obtain the "price of other" and "share of other" subsidiary firm
        other_price = other_firms['guess_price'].values[0]
        other_share = other_firms['s_jt'].values[0]
        other_mc = other_firms[mc].values[0]

        # Return the updated price
        return row[mc] + ((1 - phi_hat)/(alpha_hat * (1 - phi_hat * row['s_jt_wg'] - (1 - phi_hat) * row['s_jt']))) * (1 + alpha_hat * other_share * (other_price - other_mc) * (1 + (phi_hat * row['s_jt_wg'])/((1 - phi_hat)*row['s_jt'])))
    elif row['firm'] in ['BB', 'BB2']:
        # Filter the DataFrame for the same market as the current row
        same_market = df[df['market'] == row['market']]
        # Identify the "other" firm(s) -- subsidiary firm in the same market
        other_firms = same_market[(same_market['firm'] != row['firm']) & (same_market['firm'].isin(['BB', 'BB2']))]

        # obtain the "price of other" and "share of other" subsidiary firm
        other_price = other_firms['guess_price'].values[0]
        other_share = other_firms['s_jt'].values[0]
        other_mc = other_firms[mc].values[0]

        # Return the updated price
        return row[mc] + ((1 - phi_hat)/(alpha_hat * (1 - phi_hat * row['s_jt_wg'] - (1 - phi_hat) * row['s_jt']))) * (1 + alpha_hat * other_share * (other_price - other_mc) * (1 + (phi_hat * row['s_jt_wg'])/((1 - phi_hat)*row['s_jt'])))
    else:
        raise ValueError("Unexpected firm value")
    

### Function to calculate shares (overall market shares and within group market share):
def share(df, bl, bc, bs, ba, bb, alpha_hat, phi_hat):
    delta_jt = bl * df['isLocal'] + bc * df['isCorporate'] + bs * df['isSubsidiary'] + ba * df['alcohol'] + bb * df['bitterness'] - alpha_hat * df['updated_price'] + df['xi_j']
    df['num'] = np.exp(delta_jt/(1 - phi_hat)) # numerator for sbar_j|g (within group)
    df['den_wg'] = df.groupby(['market', 'isLocal'])['num'].transform('sum') # denominator for sbar_j|g (within group)
    # now for the denominator for s_jt (whole market share)
    # Initialize the 'dg' column
    df['dg'] = 0.0
    # Loop over each market and calculate the 'dg' value
    for market in df['market'].unique():
        # Get the values for isLocal == 1 and isLocal == 0
        local_value = df[(df['market'] == market) & (df['isLocal'] == 1)]['den_wg'].iloc[0]
        non_local_value = df[(df['market'] == market) & (df['isLocal'] == 0)]['den_wg'].iloc[0]
        # Apply the formula and assign to the 'dg' column for all rows in the market
        dg_value = 1 + local_value**(1 - phi_hat) + non_local_value**(1 - phi_hat)
    
        # Update 'dg' for the entire market
        df.loc[df['market'] == market, 'dg'] = dg_value
    
    # market share for each firm in each market
    df['s_jt'] = df['num']/(df['den_wg']**phi_hat * df['dg'])

    #within group market share for each firm in each market
    df['s_jt_wg'] = df['num']/df['den_wg']
    return df


In [10]:
# Coefficients from estimated model
alpha_hat = np.abs(result_iv.params['updated_price']) # coefficient on price
phi_hat = np.abs(result_iv.params['log_sjt_wg'])# nesting parameter
bl = np.abs(result_iv.params['isLocal']) # coefficient on local
bs = np.abs(result_iv.params['isSubsidiary'])# coefficient on subsidiary
bc = np.abs(result_iv.params['isCorporate'])# coefficient on corporate
ba = np.abs(result_iv.params['alcohol'])# coefficient on alcohol
bb = np.abs(result_iv.params['bitterness'])# coefficient on bitterness
##############################################################################################################################
########################################### Assuming BB2 did not adopt BB's cost structure ###################################
##############################################################################################################################
# Initial Guess Prices
data_buyout['guess_price'] = data_buyout['mc_jt'].copy()
data_buyout['updated_price'] = data_buyout['mc_jt'].copy()

# Tolerance for convergence
tolerance = 1e-6
max_iterations = 1000

# Iterative convergence process
for iteration in range(max_iterations):
    # calculate shares for each updates
    df = share(data_buyout, bl, bc, bs, ba, bb, alpha_hat, phi_hat)

    #calculate updated price
    df['updated_price'] = df.apply(lambda row: calculate_new_price(row, df, alpha_hat, phi_hat, 'mc_jt'), axis = 1)

    # Check for convergence
    price_difference = np.abs(df['updated_price'] - df['guess_price'])
    # update price for next iteration
    df['guess_price'] = df['updated_price']
    
    if price_difference.max() < tolerance:
        print(f"Converged: {iteration + 1} iterations")
        break
    else:
        print(f"{iteration+1} iterations")

# Final dataframe and market shares after price convergence
data_buyout = share(df, bl, bc, bs, ba, bb, alpha_hat, phi_hat)




1 iterations
2 iterations
3 iterations
4 iterations
5 iterations
6 iterations
7 iterations
8 iterations
9 iterations
10 iterations
11 iterations
12 iterations
13 iterations
14 iterations
15 iterations
16 iterations
17 iterations
18 iterations
19 iterations
20 iterations
21 iterations
22 iterations
23 iterations
24 iterations
25 iterations
26 iterations
27 iterations
28 iterations
29 iterations
30 iterations
31 iterations
32 iterations
33 iterations
34 iterations
35 iterations
36 iterations
Converged: 37 iterations


In [ ]:
# Summary Statistics Assuming BB2 did not adopt BB's cost structure
bb2_prices = data_buyout[data_buyout['firm'] == 'BB2']['updated_price']
bb_prices = data_buyout[data_buyout['firm'] == 'BB']['updated_price']
bb2_market1_prices = data_buyout[(data_buyout['firm'] == 'BB2') & (data_buyout['market'] == 1)]['updated_price']
bb_market1_prices = df[(data_buyout['firm'] == 'BB') & (data_buyout['market'] == 1)]['updated_price']
print("BB2 did not adopt BB's cost structure")
print(f'\n Summary Statistics of BB - All Market \n {bb_prices.describe()}')
print(f'Summary Statistics of BB2 - All Market \n {bb2_prices.describe()}')

print(f'Summary Statistics of BB - Market 1 \n {bb_market1_prices.describe()}')
print(f'Summary Statistics of BB2 - Market 1 \n {bb2_market1_prices.describe()}')

Summary Statistics of BB - All Market 
 count    196.000000
mean       2.106504
std        0.129073
min        1.826299
25%        2.012739
50%        2.089154
75%        2.180335
max        2.647572
Name: updated_price, dtype: float64
Summary Statistics of BB2 - All Market 
 count    196.000000
mean       2.304085
std        0.144749
min        1.982204
25%        2.211030
50%        2.310773
75%        2.396417
max        2.707678
Name: updated_price, dtype: float64
Summary Statistics of BB - Market 1 
 count    1.00000
mean     2.02788
std          NaN
min      2.02788
25%      2.02788
50%      2.02788
75%      2.02788
max      2.02788
Name: updated_price, dtype: float64
Summary Statistics of BB2 - Market 1 
 count    1.000000
mean     2.178571
std           NaN
min      2.178571
25%      2.178571
50%      2.178571
75%      2.178571
max      2.178571
Name: updated_price, dtype: float64


In [12]:
##############################################################################################################################
########################################### Assuming BB2 adopts BB's cost structure ##########################################
##############################################################################################################################
# Initial Guess Prices
data_buyout['guess_price'] = data_buyout['mc_jt'].copy()
data_buyout['updated_price'] = data_buyout['mc_jt'].copy()

# Tolerance for convergence
tolerance = 1e-6
max_iterations = 1000

# Iterative convergence process
for iteration in range(max_iterations):
    # calculate shares for each updates
    df = share(data_buyout, bl, bc, bs, ba, bb, alpha_hat, phi_hat)

    #calculate updated price
    df['updated_price'] = df.apply(lambda row: calculate_new_price(row, df, alpha_hat, phi_hat, 'mc_jt1'), axis = 1)

    # Check for convergence
    price_difference = np.abs(df['updated_price'] - df['guess_price'])
    # update price for next iteration
    df['guess_price'] = df['updated_price']
    
    if price_difference.max() < tolerance:
        print(f"Converged: {iteration + 1} iterations")
        break
    else:
        print(f"{iteration+1} iterations")

# Final dataframe and market shares after price convergence
data_buyout = share(df, bl, bc, bs, ba, bb, alpha_hat, phi_hat)

1 iterations
2 iterations
3 iterations
4 iterations
5 iterations
6 iterations
7 iterations
8 iterations
9 iterations
10 iterations
11 iterations
12 iterations
13 iterations
14 iterations
15 iterations
16 iterations
17 iterations
18 iterations
19 iterations
20 iterations
21 iterations
22 iterations
23 iterations
24 iterations
25 iterations
26 iterations
27 iterations
28 iterations
29 iterations
30 iterations
31 iterations
32 iterations
33 iterations
34 iterations
35 iterations
Converged: 36 iterations


In [14]:
# Summary Statistics Assuming BB2 adopt BB's cost structure
bb2_prices = data_buyout[data_buyout['firm'] == 'BB2']['updated_price']
bb_prices = data_buyout[data_buyout['firm'] == 'BB']['updated_price']
bb2_market1_prices = data_buyout[(data_buyout['firm'] == 'BB2') & (data_buyout['market'] == 1)]['updated_price']
bb_market1_prices = df[(data_buyout['firm'] == 'BB') & (data_buyout['market'] == 1)]['updated_price']
print("BB2 adopt BB's cost structure")
print(f'\n Summary Statistics of BB - All Market \n {bb_prices.describe()}')
print(f'Summary Statistics of BB2 - All Market \n {bb2_prices.describe()}')

print(f'Summary Statistics of BB - Market 1 \n {bb_market1_prices.describe()}')
print(f'Summary Statistics of BB2 - Market 1 \n {bb2_market1_prices.describe()}')

BB2 adopt BB's cost structure

 Summary Statistics of BB - All Market 
 count    196.000000
mean       2.134557
std        0.130406
min        1.849729
25%        2.038883
50%        2.117247
75%        2.208803
max        2.681531
Name: updated_price, dtype: float64
Summary Statistics of BB2 - All Market 
 count    196.000000
mean       2.130756
std        0.116809
min        1.862610
25%        2.054628
50%        2.128383
75%        2.204547
max        2.466526
Name: updated_price, dtype: float64
Summary Statistics of BB - Market 1 
 count    1.000000
mean     2.053229
std           NaN
min      2.053229
25%      2.053229
50%      2.053229
75%      2.053229
max      2.053229
Name: updated_price, dtype: float64
Summary Statistics of BB2 - Market 1 
 count    1.000000
mean     2.031924
std           NaN
min      2.031924
25%      2.031924
50%      2.031924
75%      2.031924
max      2.031924
Name: updated_price, dtype: float64


In [24]:
# Summary Statistics pre-buyout
data = rc.data
bb2_prices = data[data['firm'] == 'local_1']['updated_price']
bb_prices = data[data['firm'] == 'BB']['updated_price']
bb2_market1_prices = data[(data['firm'] == 'local_1') & (data['market'] == 1)]['updated_price']
bb_market1_prices = data[(data['firm'] == 'BB') & (data['market'] == 1)]['updated_price']
print("Pre-buyout")
print(f'\n Summary Statistics of BB - All Market \n {bb_prices.describe()}')
print(f'Summary Statistics of BB2 - All Market \n {bb2_prices.describe()}')

print(f'Summary Statistics of BB - Market 1 \n {bb_market1_prices.describe()}')
print(f'Summary Statistics of BB2 - Market 1 \n {bb2_market1_prices.describe()}')

Pre-buyout

 Summary Statistics of BB - All Market 
 count    196.000000
mean       1.984290
std        0.122906
min        1.737138
25%        1.902570
50%        1.972652
75%        2.060917
max        2.519862
Name: updated_price, dtype: float64
Summary Statistics of BB2 - All Market 
 count    25.000000
mean      2.291569
std       0.080236
min       2.167755
25%       2.239418
50%       2.280771
75%       2.327130
max       2.485547
Name: updated_price, dtype: float64
Summary Statistics of BB - Market 1 
 count    1.000000
mean     1.851065
std           NaN
min      1.851065
25%      1.851065
50%      1.851065
75%      1.851065
max      1.851065
Name: updated_price, dtype: float64
Summary Statistics of BB2 - Market 1 
 count    1.000000
mean     2.167755
std           NaN
min      2.167755
25%      2.167755
50%      2.167755
75%      2.167755
max      2.167755
Name: updated_price, dtype: float64
